##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Function calling config

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Function_calling_config.ipynb"><img src="https://github.com/google-gemini/cookbook/blob/main/images/colab_logo_32px.png?raw=1" />Run in Google Colab</a>
  </td>
</table>

Specifying a `function_calling_config` allows you to control how the Gemini API acts when `tools` have been specified. For example, you can choose to only allow free-text output (disabling function calling), force it to choose from a subset of the functions provided in `tools`, or let it act automatically.

This guide assumes you are already familiar with function calling. For an introduction, check out the [docs](https://ai.google.dev/docs/function_calling).

In [ ]:
!pip install -U -q "google-generativeai>=0.7.2"

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

## Set up a model with tools

This example uses 3 functions that control a simple hypothetical EMAILBOT system. Using these functions requires them to be called in a specific order. For example, you must turn_on_laptop before you can send email.

While you can pass these directly to the model and let it try to call them correctly, specifying the `function_calling_config` gives you precise control over the functions that are available to the model.

In [ ]:
def turn_on_laptop():
    """Power on the laptop."""
    print("SYSTEM: Laptop has been powered on.")


def send_email(email: str):
    """Send an email to the specified email address."""
    print(f"EMAILBOT: Email sent to {email}.")


def turn_off_laptop():
    """Power off the laptop."""
    print("SYSTEM: Laptop has been powered off.")


device_controls = [turn_on_laptop, send_email, turn_off_laptop]
instruction = """You are a helpful device control bot.You can
                 Turn on the laptop, send an email, and then turn off the laptop.Do not perform any other tasks."""


model = genai.GenerativeModel(
    "models/gemini-1.5-pro",
    tools=device_controls,
    system_instruction=instruction
)

chat = model.start_chat()

Create a helper function for setting `function_calling_config` on `tool_config`.

In [ ]:
from google.generativeai.types import content_types
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

## Text-only mode: `NONE`

If you have provided the model with tools, but do not want to use those tools for the current conversational turn, then specify `NONE` as the mode. `NONE` tells the model not to make any function calls, and will behave as though none have been provided.

In [ ]:
tool_config = tool_config_from_mode("none")


In [ ]:
response = chat.send_message(
    "Hello device-control bot, what tasks can you perform?", tool_config=tool_config
)


In [ ]:
print(response.text)

I can turn on the laptop, send an email, and then turn off the laptop.



## Automatic mode: `AUTO`

To allow the model to decide whether to respond in text or call specific functions, you can specify `AUTO` as the mode.

In [ ]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Turn on the laptop, send an email to a shirazkk8@gmail.com, and then turn it off.", tool_config=tool_config)

In [ ]:
response.parts

[function_call {
  name: "turn_on_laptop"
  args {
  }
}
, function_call {
  name: "send_email"
  args {
    fields {
      key: "email"
      value {
        string_value: "shirazkk8@gmail.com"
      }
    }
  }
}
, function_call {
  name: "turn_off_laptop"
  args {
  }
}
]

In [ ]:
chat.rewind();  # You are not actually calling the function, so remove this from the history.

## Function-calling mode: `ANY`

Setting the mode to `ANY` will force the model to make a function call. By setting `allowed_function_names`, the model will only choose from those functions. If it is not set, all of the functions in `tools` are candidates for function calling.


In [ ]:
print(response.parts[0])

function_call {
  name: "turn_on_laptop"
  args {
  }
}



In [ ]:
available_fns = ["send_email", "turn_off_laptop"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("send an email to shirazali22@gmail.com", tool_config=tool_config)

In [ ]:
print(response.parts)

[function_call {
  name: "send_email"
  args {
    fields {
      key: "email"
      value {
        string_value: "shirazali22@gmail.com"
      }
    }
  }
}
]


## Automatic function calling

`tool_config` works when enabling automatic function calling too.

In [ ]:
# available_fns = ["enable_lights"]
available_fns = ["turn_on_laptop", "send_email", "turn_off_laptop"]
tool_config = tool_config_from_mode("any", available_fns)

In [ ]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
res = auto_chat.send_message("Power on the laptop, send an email to a alijawwad33@gmail.com, and then shut it down.", tool_config=tool_config)

SYSTEM: Laptop has been powered on.
EMAILBOT: Email sent to alijawwad33@gmail.com.
SYSTEM: Laptop has been powered off.


In [ ]:
auto_chat = model.start_chat(enable_automatic_function_calling=True)
res = auto_chat.send_message("Turn off the laptop", tool_config=tool_config)

SYSTEM: Laptop has been powered off.


In [ ]:
def set_laptop_power_settings(brightness: int, power_mode: str):
    """Set the laptop screen brightness and power mode. (mock API).

    Args:
        brightness (int): Screen brightness level from 0 to 100.
                          Zero is the lowest brightness, and 100 is the highest.
        power_mode (str): Power mode of the laptop, which can be `performance`, `balanced`, or `power_saver`.

    Returns:
        dict: A dictionary containing the set brightness and power mode.
    """
    return {
        "screenBrightness": brightness,
        "powerMode": power_mode
    }


In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_laptop_power_settings])

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message("It's nighttime, and I want to work comfortably without straining my eyes. Adjust the laptop settings accordingly.")
response.text

"OK. I've set your laptop's brightness to 30 and the power mode to balanced.  Let me know if you need any further adjustments.\n"

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message(
    "It's daytime, and I need a bright screen for better visibility while working. Adjust the laptop settings accordingly."
)
response.text


'OK. I\'ve set the screen brightness to 100 and the power mode to "performance" for optimal daytime visibility.\n'

In [ ]:
from typing import Literal

def set_laptop_display(brightness: int, mode: Literal["daylight", "cool", "warm"]):
    """Adjust the laptop screen brightness and display color mode. (mock API).

    Args:
        brightness (int): Screen brightness level from 0 to 100.
                          Zero is the lowest brightness, and 100 is the highest.
        mode (str): Display color mode, which can be:
                    - `daylight`: Bright and natural for outdoor or daytime work.
                    - `cool`: Cooler tones, reducing eye strain for prolonged use.
                    - `warm`: Warmer tones for a cozy and comfortable display.

    Returns:
        dict: A dictionary containing the adjusted brightness and display color mode.
    """
    return {
        "screenBrightness": brightness,
        "displayMode": mode
    }

# Integrating the function into Google's AI model
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_laptop_display])

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message("It's daylight. Adjust my laptop display for a daytime work.")
print(response.text)


OK. I've adjusted your laptop display to daylight mode with 75 brightness.


In [ ]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [ ]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [ ]:
print(x := 10)  # Assigns AND prints x

10


In [ ]:
def control_air_conditioner(on: bool, temperature: int) -> bool:
    """Controls the air conditioning system.

    Args:
        on (bool): Whether to turn the AC on or off.
        temperature (int): Desired room temperature.

    Returns:
        bool: Confirmation that the action was executed.
    """
    print(f"Air conditioning is {'ON' if on else 'OFF'}, set to {temperature}°C.")
    return True


def play_relaxing_music(stressed: bool) -> str:
    """Plays relaxing music if the user is stressed.

    Args:
        stressed (bool): Whether the user is feeling stressed.

    Returns:
        str: The name of the song being played.
    """
    if stressed:
        song = "Calm Waves - Meditation Playlist"
        print(f"Playing relaxing music: {song}")
    else:
        song = "No music needed."
        print("No music is playing.")
    return song


def adjust_room_lighting(mood: str) -> bool:
    """Adjusts lighting based on mood.

    Args:
        mood (str): The user's mood, can be 'relaxed', 'energetic', or 'neutral'.

    Returns:
        bool: Confirmation that the action was executed.
    """
    lighting_settings = {
        "relaxed": "Dim, warm light",
        "energetic": "Bright, cool light",
        "neutral": "Balanced white light"
    }
    setting = lighting_settings.get(mood, "Neutral white light")
    print(f"Lights set to: {setting}")
    return True


In [ ]:
# Register functions as tools for AI
smart_home_tools = [control_air_conditioner, play_relaxing_music, adjust_room_lighting]

# Initialize AI Model with tools
model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=smart_home_tools)

# Start an interactive AI chat
chat = model.start_chat()


In [ ]:
# User requests a comfortable atmosphere
response = chat.send_message("I'm feeling stressed, and it's really hot in here. Make things comfortable.")

# Print out each function call the AI decides to make
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")


play_relaxing_music(stressed=True)
control_air_conditioner(temperature=25.0, on=True)


In [ ]:
model._tools.to_proto()

[function_declarations {
   name: "power_disco_ball"
   description: "Powers the spinning disco ball."
   parameters {
     type_: OBJECT
     properties {
       key: "power"
       value {
         type_: BOOLEAN
       }
     }
     required: "power"
   }
 }
 function_declarations {
   name: "start_music"
   description: "Play some music matching the specified parameters.\n\n    Args:\n      energetic: Whether the music is energetic or not.\n      loud: Whether the music is loud or not.\n      bpm: The beats per minute of the music.\n\n    Returns: The name of the song being played.\n    "
   parameters {
     type_: OBJECT
     properties {
       key: "loud"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "energetic"
       value {
         type_: BOOLEAN
       }
     }
     properties {
       key: "bpm"
       value {
         type_: INTEGER
       }
     }
     required: "energetic"
     required: "loud"
     required: "bpm"
   }
 }
 functi

## Further reading

Check out the function calling [quickstart](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb) for an introduction to function calling. You can find another fun function calling example [here](https://github.com/google-gemini/cookbook/blob/main/quickstarts/rest/Function_calling_REST.ipynb) using curl.
